In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from PIL import *
from google.colab import drive
X_data = []
Y_data = []


In [ ]:

datadir = "/content/drive/MyDrive/dataset/casia2"
tp = ["Tp","Tp_gt"]
INPUT_SIZE = 224
from albumentations import (
PadIfNeeded,
HorizontalFlip,
VerticalFlip,
Transpose,
HueSaturationValue,
ElasticTransform,
GridDistortion,
OpticalDistortion,
RandomBrightnessContrast,
RandomGamma,Resize
)
def agument(aug,image,mask):
    #This function will apply the type of augmentation on both image and mask
    #returns augmented image and its augmented mask
    augmented = aug(image=image, mask=mask)
    return augmented['image'],augmented['mask']


# All these below functions defines the different agumentations types.
def elasticTransform(image,mask):
    aug = ElasticTransform(p=1)
    return agument(aug,image,mask)

def opticalDistortion(image,mask):
    aug = OpticalDistortion(p=1, distort_limit=3, shift_limit=0.4)
    return agument(aug,image,mask)

def randomBrightnessContrast(image,mask):
    aug =RandomBrightnessContrast(p=1,brightness_limit=0.5, contrast_limit=0.4)
    return agument(aug,image,mask)


In [ ]:
tp_paths = os.listdir(os.path.join(datadir,tp[0])) #paths for tampered and groundtruths
gt_paths = os.listdir(os.path.join(datadir,tp[1]))
tp_paths.sort() #alphabetically sort the data such that img and mask can be ound easily
gt_paths.sort()
aug_x = []
aug_y = []
i = 1

#now for all the images we will convert them and make them into a data file 
for i in range(0,700):
    img_arr = cv2.imread(os.path.join(datadir,tp[0],tp_paths[i]),cv2.IMREAD_COLOR)/255
    mask_arr = cv2.imread(os.path.join(datadir,tp[1],gt_paths[i]),cv2.IMREAD_GRAYSCALE)/255
    img_input = cv2.resize(img_arr,(INPUT_SIZE,INPUT_SIZE))
    mask_output = cv2.resize(mask_arr,(208,208))
    #image augmentations and attaching them
    
    image_0 = img_input
    mask_0 = mask_output  
    aug_x.append(image_0)
    aug_y.append(mask_0)
    
    image_1,mask_1= elasticTransform(image=image_0,mask=mask_0)
    aug_x.append(image_1)
    aug_y.append(mask_1)
    
    image_2,mask_2= opticalDistortion(image=image_0,mask=mask_0)
    aug_x.append(image_2)
    aug_y.append(mask_2)
      
    image_3,mask_3= randomBrightnessContrast(image=image_0,mask=mask_0)
    aug_x.append(image_3)
    aug_y.append(mask_3)
    
    print(i)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: ignored

In [ ]:
#print(len(X_data),len(Y_data))
X = np.array(aug_x)
Y = np.array(aug_y)
#np.save("/content/drive/MyDrive/dataset/CASIA 1.0 dataset/X.npy",X)

#np.save("/content/drive/MyDrive/dataset/CASIA 1.0 dataset/Y.npy",Y)
X.reshape(-1,224,224,3)
Y.reshape(-1,208,208,1)
print(X.shape)
print(X.shape)


(1433, 224, 224, 3)
(1433, 224, 224, 3)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import datetime
import tensorflow as tf
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.preprocessing.image import img_to_array
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D ,Add 
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint ,TensorBoard
from keras.layers import Input,  Dropout,Cropping2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs

basemodel = VGG16(input_tensor=Input(shape=(224,224,3)),include_top=False,weights='imagenet')
#basemodel.summary()


#conv block 1
conv_1_1 = basemodel.get_layer('block1_conv1').output
batchnorm_1_1 = BatchNormalization()(conv_1_1)
relu_1_1 = Activation('relu')(batchnorm_1_1)

conv_1_2 =  (basemodel.get_layer('block1_conv2'))(relu_1_1)
batchnorm_1_2 = BatchNormalization()(conv_1_2)
relu_1_2 = Activation('relu')(batchnorm_1_2)
maxpool_1 = (basemodel.get_layer('block1_pool'))(relu_1_2)

#conv block 2

conv_2_1 = (basemodel.get_layer('block2_conv1'))(maxpool_1)
batchnorm_2_1 = BatchNormalization()(conv_2_1)
relu_2_1 = Activation('relu')(batchnorm_2_1)

conv_2_2 =  (basemodel.get_layer('block2_conv2'))(relu_2_1)
batchnorm_2_2 = BatchNormalization()(conv_2_2)
relu_2_2 = Activation('relu')(batchnorm_2_2)
maxpool_2 = (basemodel.get_layer('block2_pool'))(relu_2_2)


#conv block 3

conv_3_1 = (basemodel.get_layer('block3_conv1'))(maxpool_2)
batchnorm_3_1 = BatchNormalization()(conv_3_1)
relu_3_1 = Activation('relu')(batchnorm_3_1)

conv_3_2 =  (basemodel.get_layer('block3_conv2'))(relu_3_1)
batchnorm_3_2 = BatchNormalization()(conv_3_2)
relu_3_2 = Activation('relu')(batchnorm_3_2)

conv_3_3 =  (basemodel.get_layer('block3_conv3'))(relu_3_2)
batchnorm_3_3 = BatchNormalization()(conv_3_3)
relu_3_3 = Activation('relu')(batchnorm_3_3)

maxpool_3 = (basemodel.get_layer('block3_pool'))(relu_3_3)

# skip connections from maxpool 3 introduced
Skip_3 = maxpool_3
Skip_3 = Conv2D(2,1)(Skip_3)
Skip_3 = Cropping2D()(Skip_3)


#conv block 4

conv_4_1 = (basemodel.get_layer('block4_conv1'))(maxpool_3)
batchnorm_4_1 = BatchNormalization()(conv_4_1)
relu_4_1 = Activation('relu')(batchnorm_4_1)

conv_4_2 =  (basemodel.get_layer('block4_conv2'))(relu_4_1)
batchnorm_4_2 = BatchNormalization()(conv_4_2)
relu_4_2 = Activation('relu')(batchnorm_4_2)

conv_4_3 =  (basemodel.get_layer('block4_conv3'))(relu_4_2)
batchnorm_4_3 = BatchNormalization()(conv_4_3)
relu_4_3 = Activation('relu')(batchnorm_4_3)

maxpool_4 = (basemodel.get_layer('block4_pool'))(relu_4_3)

#skip connections from  maxpool 4 introduced

Skip_4 = maxpool_4
Skip_4 = Conv2D(2,1)(Skip_4)
Skip_4 = Cropping2D()(Skip_4)

#conv block 5

conv_5_1 = (basemodel.get_layer('block5_conv1'))(maxpool_4)
batchnorm_5_1 = BatchNormalization()(conv_5_1)
relu_5_1 = Activation('relu')(batchnorm_5_1)

conv_5_2 =  (basemodel.get_layer('block5_conv2'))(relu_5_1)
batchnorm_5_2 = BatchNormalization()(conv_5_2)
relu_5_2 = Activation('relu')(batchnorm_5_2)

conv_5_3 =  (basemodel.get_layer('block5_conv3'))(relu_5_2)
batchnorm_5_3 = BatchNormalization()(conv_5_3)
relu_5_3 = Activation('relu')(batchnorm_5_3)

maxpool_5 = (basemodel.get_layer('block5_pool'))(relu_5_3)


#conv 6-7 layers 


forward = Conv2D(4096,7,padding='same')(maxpool_5)
forward = Activation('relu')(forward)
forward = Dropout(0.5)(forward)

forward = Conv2D(4096,1,padding='same')(forward)
forward = Activation('relu')(forward)
forward = Dropout(0.5)(forward)
  
forward = Conv2D(2,1,padding='same')(forward)
forward =  UpSampling2D(size=(2,2))(forward)
#forward = Conv2DTranspose(2, (8,8), kernel_initializer='glorot_uniform', padding='valid', use_bias=False)(forward)



#deconv surface mask layer

surfaceoutlayer = Add()([Skip_4,forward])
#edgeoutlayer = Add()([Skip_4,forward])
surfaceoutlayer = UpSampling2D(size=(2,2))(surfaceoutlayer)
surfaceoutlayer = Add()([surfaceoutlayer,Skip_3])
surfaceoutlayer = UpSampling2D(size=(8,8))(surfaceoutlayer)
surfaceoutlayer = Cropping2D((8,8))(surfaceoutlayer)

surfaceoutlayer = Activation('softmax')(surfaceoutlayer)
surfaceoutlayer = Conv2D(1,1)(surfaceoutlayer)


#model test 
MFCN = Model(inputs=basemodel.input,outputs=(surfaceoutlayer),name="MODIFIED_VGG_MODEL")
MFCN.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
MFCN.summary()


Model: "MODIFIED_VGG_MODEL"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 224, 224, 64) 256         block1_conv1[0][0]               
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 224, 224, 64) 0           batch_normalization_13[0][0]     
_________________________________________________________________________________

In [ ]:
import tensorflow as tf
"""
from albumentations import (
PadIfNeeded,
HorizontalFlip,
VerticalFlip,
Transpose,
HueSaturationValue,
ElasticTransform,
GridDistortion,
OpticalDistortion,
RandomBrightnessContrast,
RandomGamma,Resize
)
def agument(aug,image,mask):
    #This function will apply the type of augmentation on both image and mask
    #returns augmented image and its augmented mask
    augmented = aug(image=image, mask=mask)
    return augmented['image'],augmented['mask']


# All these below functions defines the different agumentations types.

def horizontalFlip(image,mask):
    aug = HorizontalFlip(p=1)
    return agument(aug,image,mask)
    
def verticalFlip(image,mask):
    aug = VerticalFlip(p=1)
    return agument(aug,image,mask)

def transpose(image,mask):
    aug = Transpose(p=1)
    return agument(aug,image,mask) 

def elasticTransform(image,mask):
    aug = ElasticTransform(p=1)
    return agument(aug,image,mask)

def opticalDistortion(image,mask):
    aug = OpticalDistortion(p=1, distort_limit=3, shift_limit=0.4)
    return agument(aug,image,mask)

def randomBrightnessContrast(image,mask):
    aug =RandomBrightnessContrast(p=1,brightness_limit=0.5, contrast_limit=0.4)
    return agument(aug,image,mask)

X_train = []
Y_train = []
p = len(aug_x)-1
for i in range(0,p):
  image_0 = aug_x[i]  
  mask_0 = aug_y[i]
  np.append(aug_x,image_0)
  np.append(aug_y,mask_0)
  image_1,mask_1= horizontalFlip(image=image_0,mask=mask_0)
  np.append(aug_x,image_1)
  np.append(aug_y,mask_1)
  image_2,mask_2= verticalFlip(image=image_0,mask=mask_0)
  np.append(aug_x,image_2)
  np.append(aug_y,mask_2)
  image_3,mask_3= transpose(image=image_0,mask=mask_0)
  np.append(aug_x,image_3)
  np.append(aug_y,mask_3)
  image_4,mask_4= elasticTransform(image=image_0,mask=mask_0)
  np.append(aug_x,image_4)
  np.append(aug_y,mask_4)
  image_5,mask_5= opticalDistortion(image=image_0,mask=mask_0)
  np.append(aug_x,image_5)
  np.append(aug_y,mask_5)
  image_6,mask_6= randomBrightnessContrast(image=image_0,mask=mask_0)
  np.append(aug_x,image_6)
  np.append(aug_y,mask_6)
 
"""

"\nfrom albumentations import (\nPadIfNeeded,\nHorizontalFlip,\nVerticalFlip,\nTranspose,\nHueSaturationValue,\nElasticTransform,\nGridDistortion,\nOpticalDistortion,\nRandomBrightnessContrast,\nRandomGamma,Resize\n)\ndef agument(aug,image,mask):\n    #This function will apply the type of augmentation on both image and mask\n    #returns augmented image and its augmented mask\n    augmented = aug(image=image, mask=mask)\n    return augmented['image'],augmented['mask']\n\n\n# All these below functions defines the different agumentations types.\n\ndef horizontalFlip(image,mask):\n    aug = HorizontalFlip(p=1)\n    return agument(aug,image,mask)\n    \ndef verticalFlip(image,mask):\n    aug = VerticalFlip(p=1)\n    return agument(aug,image,mask)\n\ndef transpose(image,mask):\n    aug = Transpose(p=1)\n    return agument(aug,image,mask) \n\ndef elasticTransform(image,mask):\n    aug = ElasticTransform(p=1)\n    return agument(aug,image,mask)\n\ndef opticalDistortion(image,mask):\n    aug =

In [ ]:
es_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
tb_cb = tf.keras.callbacks.TensorBoard('./logs', update_freq=1)
history = MFCN.fit(X,Y,batch_size=20,epochs=5,verbose=1,callbacks=[es_cb,tb_cb])
MFCN.save("/content/drive/MyDrive/dataset/my_model_2.h5")

Epoch 1/5
72/72 [==============================] - 3505s 49s/step - loss: 2.0684 - accuracy: 0.8633
Epoch 2/5
72/72 [==============================] - 3509s 49s/step - loss: 2.0776 - accuracy: 0.8627
Epoch 3/5
72/72 [==============================] - 3529s 49s/step - loss: 1.9932 - accuracy: 0.8682
Epoch 4/5
72/72 [==============================] - 3550s 49s/step - loss: 2.0472 - accuracy: 0.8647
Epoch 5/5
72/72 [==============================] - 3556s 49s/step - loss: 2.0885 - accuracy: 0.8620


In [ ]:
from tensorflow.keras.models import load_model
def print_result():
  model_path="/content/drive/MyDrive/dataset/my_model_2.h5"
  model = load_model(model_path)
  sample_path = '/content/drive/MyDrive/dataset/CASIA 1.0 dataset/Tp/Sp/Sp_D_CNN_A_pla0030_pla0061_0272.jpg'
  GT_mask ='/content/drive/MyDrive/dataset/CASIA 1.0 dataset/Tp_gt/Sp/Sp_D_CNN_A_pla0030_pla0061_0272_gt.jpg'
  

  image = cv2.imread(sample_path,cv2.IMREAD_COLOR)/255
  image_arr = cv2.resize(image,(224,224))
  Ground_Truth = cv2.imread(GT_mask)

  plt.imshow(image_arr)
  plt.show()
  plt.imshow(Ground_Truth)
  plt.show()  
  print(image_arr.shape)
  img = np.array(image_arr).reshape(1,224,224,3)                
  Predicted_mask = model.predict(img)
  pm = np.array(Predicted_mask).reshape(208,208)
  plt.imshow(pm)
  plt.show()


In [ ]:
print_result()

RuntimeError: ignored